In [1]:
import json
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas
from pathlib import Path
import os

The steps:
1. Get a list of the ids
2. Get a list of image id's pointing to images ls


Make sure your cwd is in /notebooks/preprocessing/. 

In [1]:
file = open(Path("../original.json"))
original = json.load(file)
path = Path("../../../data/cropped/")
try:
    path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
else:
    print("Folder was created")

NameError: name 'Path' is not defined

In [3]:
id_list = []
same_images = []

for entry in original["annotations"]:
    if entry['id'] not in id_list:
        id_list.append(entry['id'])

#populate ph with image id's that has more than 1 image connected to it!
ph =[]
for entry in original["annotations"]:
    for entry2 in original["annotations"]:
        if entry2['image_id'] == entry['image_id'] and entry['id'] != entry2['id'] and entry["image_id"] not in ph:
            ph.append(entry2['image_id'])
        
for _ in ph:
    tmp = [_]
    same_images.append(tmp)
    

for entry in original["annotations"]:
    im_id = entry['image_id']
    id = entry['id']
    for list in same_images:
        if im_id in list and id not in list:
            list.append(id)


print(r"Length of image id's {}".format(len(same_images)))

Length of image id's 130


In [4]:
#The list same images will hold lists that contain (image_id, all other id's pertaining to this image id)
print(same_images)


[[7073954, 504081, 504082], [7036543, 504300, 504301], [7026503, 504147, 504148], [7025381, 504093, 504094], [7036076, 504284, 504285], [7025641, 504115, 504116], [7048995, 504480, 504481], [7035006, 504260, 504261, 504262], [7049171, 504486, 504487, 504488], [7060866, 504632, 504633], [7041047, 504400, 504401], [7070717, 504784, 504785, 504786], [7038349, 504355, 504356], [7070122, 504766, 504767], [7074046, 504084, 504085], [7054613, 504022, 504023], [7038293, 504335, 504336], [7034548, 504239, 504240], [7068630, 504718, 504719], [7071722, 504838, 504839], [7038251, 504328, 504329], [7038267, 504331, 504332], [7038425, 504362, 504363], [7034714, 504246, 504247], [7072864, 504073, 504074], [7070917, 504799, 504800, 504801], [7072506, 504855, 504856], [7041084, 504402, 504403], [7070287, 504774, 504775], [7029096, 504190, 504191], [7050345, 504012, 504013], [7035038, 504264, 504265], [7071864, 504842, 504843], [7058575, 504569, 504570], [7030895, 503994, 503995, 503996], [7072082, 5048

In [5]:
def crop_region(image: np.array, bounding_box) -> np.array:
    """

    Args:
        image: numpy type
        bounding_box: Integer

    Returns: numpy type

    """
    x_min, y_min, x_range, y_range = bounding_box

    _ = np.flipud(image[x_min:x_min + x_range, y_min:y_min + y_range])
    _ = np.rot90(_,k=3)

    return _


In [6]:
image_root = Path("..","..","data","Images")

1. Go through all the images in the same_images list
2. Recrop, if the name already exists give a suffix with the image_id

In [7]:
#Unpack to tuple for ease of use

overlap = []
for list in same_images:
    for item in list:
        # First element is image_id
        if item == list[0]:
            continue
        ################################
        # From here it'll always take id from the list
        overlap.append((list[0],item))

print(r"Count of overlap set is {}".format(len(overlap)))

#After cropping this number should match the number of entries in overlap.json

Count of overlap set is 295


In [8]:
non_overlap = []
for entry in original["annotations"]:
    if (entry["image_id"],entry["id"]) not in overlap:
        non_overlap.append((entry["image_id"], entry["id"]))
print(len(non_overlap))
print(len(overlap) + len(non_overlap))
if(len(overlap) + len(non_overlap) == len(original["annotations"])):
    print("Count of disjunct set matches original json entries")


602
897
Count of disjunct set matches original json entries


In [9]:
# Set up new json
# open JSON file
categories = original['categories']
images = original['images']
annotations = original['annotations']

# key image ids as key for dict
images_keys = [img['id'] for img in images]

# get dictionary for filenames
images_dict = dict(zip(images_keys, images))
# creat a new json file to store the new infomation

print(len(categories))
print(len(images))
print(len(annotations))


1
9585
897


In [10]:
newfile = {
    "categories": [{
        "id": 0,
        "name": "dent"
    }, {
        "id": 1,
        "name": "other"
    }, {
        "id": 2,
        "name": "rim"
    }, {
        "id": 3,
        "name": "scratch"
    }, {
        "id": 4,
        "name": "deformation"
    }],
    "images": [],
    "annotations": []
}

# Deal with non overlapping ones first

# take prelabeled labels and put them by image_id(because they are non overlap and unique)

#from 600-896 (1 to 897 in total, index from 0)
pre_den = json.load(open(Path("..", "etc", "prelabeled_den.json")))
#from 300-599
pre_mag = json.load(open(Path("..", "etc", "prelabeled_mag.json")))
#from 0-299
#pre_thu = json.load(open(Path("..", "etc", "prelabeled_thu.json")))
stitched = {"annotations": []}

for i in range(300):
    stitched["annotations"].append(pre_mag["annotations"][i])

for i in range(300,600):
    stitched["annotations"].append(pre_mag["annotations"][i])

for i in range(600,897):
    stitched["annotations"].append(pre_den["annotations"][i])

print(len(stitched["annotations"]))


897


In [11]:
count =0
relabeled_count=0
for newentry in stitched["annotations"]:
    for entry in original["annotations"]:
        if(entry["id"] == newentry["id"]):
            count+=1
            if(entry["category_id"] != newentry["category_id"]):
                relabeled_count+=1
if count == len(original["annotations"]):
    print(r"{} entries matched, valid".format(count))

print(r"{} entries were relabeled.".format(relabeled_count))

897 entries matched, valid
725 entries were relabeled.


In [12]:
problem_set = []

def get_labeled_category(id,img_id):
    for entry in stitched["annotations"]:
        if entry["id"] == id and entry["image_id"] == img_id:
            return entry["category_id"]
    print("Not found!")
    problem_set.append((id, img_id))


images_entries = original["images"]
cropped_path = Path("..", "..", "data", "cropped")


In [13]:
#only need the annotations entries since the rest are obsolete


for entry in original["annotations"]:
    if((entry["image_id"],entry["id"]) in non_overlap ):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        filename = images_dict[image_id]['file_name']
        fp = Path(image_root, filename)
        #ONLY get the category_id from prelabeled ones for NON overlap, overlap ones need relabeling, and shall be marked with category_id 1
        category_id = get_labeled_category(id,image_id)
        img = Image.open(fp)
        numpy_image = np.transpose(np.array(img), (1, 0, 2))
        cropped_image = crop_region(numpy_image, bounding_box)
        cropped_image = Image.fromarray(cropped_image)

        save_path = Path(cropped_path,str(id)+".png")
        i=0
        while(1):
            # IF NAMING using ID IS UNIQUE then there would be NO images with suffix _0 , _1 etc ...
            if Path(save_path).is_file():
                print(r"File {} already exists.".format(save_path))
                save_path = Path(cropped_path, str(id) + "_" + str(i) + ".png")
                i += 1
                continue
            break
        cropped_image.save(save_path)
        save_path = save_path.relative_to("../../data/").as_posix()
        fp = fp.relative_to("../../data").as_posix()
        # Fill out the data
        # image id refers to the source, non cropped image
        new_annotation = {
            "id": id,
            "category_id": category_id,
            'image_id': image_id,
            'filepath': save_path,
            'source': fp,
            "bbox": bounding_box
        }

        newfile['annotations'].append(new_annotation)
        
if(len(problem_set) ==0 ):
    print("No problems found.")

No problems found.


In [14]:
#only need the annotations entries since the rest are obsolete

for entry in original["annotations"]:
    if ((entry["image_id"], entry["id"]) in overlap):
        # Only access images in the list
        id = entry["id"]
        image_id = entry['image_id']
        bounding_box = entry['bbox']
        filename = images_dict[image_id]['file_name']
        fp = Path(image_root, filename)
        #ONLY get the category_id from prelabeled ones for NON overlap, overlap ones need relabeling, and shall be marked with category_id 1
        img = Image.open(fp)
        numpy_image = np.transpose(np.array(img), (1, 0, 2))
        cropped_image = crop_region(numpy_image, bounding_box)
        cropped_image = Image.fromarray(cropped_image)

        save_path = Path(cropped_path, str(id) + ".png")
        i = 0
        while (1):
            # IF NAMING using ID IS UNIQUE then there would be NO images with suffix _0 , _1 etc ...
            if Path(save_path).is_file():
                print(r"File {} already exists.".format(save_path))
                save_path = Path(cropped_path, str(id) + "_" + str(i) + ".png")
                i += 1
                continue
            break
        cropped_image.save(save_path)
        save_path = save_path.relative_to("../../data").as_posix()
        fp = fp.relative_to("../../data").as_posix()
        # Fill out the data
        # image id refers to the source, non cropped image
        new_annotation = {
            "id": id,
            "category_id": 1,
            'image_id': image_id,
            'filepath': save_path,
            'source': fp,
            "bbox": bounding_box
        }

        newfile['annotations'].append(new_annotation)


In [15]:
for entry in original["images"]:
    fp = Path(image_root,entry['file_name'])
    fp = fp.relative_to("../../data").as_posix()
    new_images = {
        "id": entry['id'],
        "filepath": fp,
        "width": entry['width'],
        "height": entry['height']
    }
    newfile['images'].append(new_images)

In [16]:
# Converting dictionaries to json
json_test = json.dumps(newfile)
with open('restructured.json', 'w') as json_file:
    json_file.write(json_test)